# $$Empirical \text{ } Finance$$
## Portfolio Returns Assignment: 
### _R&D Capital Replication_

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import statsmodels.api as sm
from patsy import dmatrices
import yfinance as yf
from datetime import datetime

from tqdm.notebook import tqdm
tqdm.pandas()

%matplotlib inline
plt.style.use("bmh")

# Helper Functions

In [2]:
def convert_to_date(date):
    return pd.to_datetime(date[:4]+"/"+date[4:6]+"/"+date[6:]).date()

# convert_to_date("19721231")

def get_rnd(xrd_list: pd.Series)->pd.Series:
    '''Function to calculate R&D investment over the last 5 years'''
    return xrd_list.rolling(window=5).apply(lambda x: (np.array([0.2, 0.4, 0.6, 0.8, 0.1]) @ np.array(x)))
# get_rnd(yearly_xrd.iloc[-1]).tail(10)

def assign_quantile(rcd_list: pd.Series)->pd.Series:
    '''Function to assign each LPERMNO to a quantile depending on its R&D Investment'''
    arr = pd.qcut(rcd_list[rcd_list != 0.0], q = 5, labels= ["Low","q2","q3","q4","High"], duplicates= "drop")
    arr = arr.cat.add_categories('N/A R&D')

    for idx in rcd_list[rcd_list == 0].index:
        arr[idx] = "N/A R&D"
    return arr

# Read CCM Fundamentals Annual

In [3]:
ccm_fundamentals_annual = pd.read_csv("df_fa.csv")

ccm_fundamentals_annual["fyear"] = ccm_fundamentals_annual["datadate"].progress_apply(lambda x: convert_to_date(str(x)).year)
ccm_fundamentals_annual["year"] = ccm_fundamentals_annual["fyear"]+1
ccm_fundamentals_annual["me"] = ccm_fundamentals_annual[["csho", "prcc_c"]].apply(lambda r: (r["csho"]*r["prcc_c"]), axis = 1)

# USD Only
ccm_fundamentals_annual = ccm_fundamentals_annual[ccm_fundamentals_annual["curcd"] == "USD"]

# Positive ME only:
ccm_fundamentals_annual = ccm_fundamentals_annual[ccm_fundamentals_annual["me"] > 0]

# SIC Codes
ccm_fundamentals_annual["sic"] = ccm_fundamentals_annual[["sic","sich"]].apply(lambda row: (row["sich"] if np.isnan(row["sic"]) else row["sic"]), axis =1)
ccm_fundamentals_annual = ccm_fundamentals_annual.drop("sich", axis = 1)

ccm_fundamentals_annual

  0%|          | 0/276007 [00:00<?, ?it/s]

,GVKEY,LPERMNO,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,cusip,curcd,csho,xrd,exchg,costat,prcc_c,prcc_f,sic,year,me
0,1000,25881,19761231,1976,INDL,C,D,STD,AE.2,000032102,USD,2.207,NaN,12,I,5.7500,5.7500,3089.0,1977,12.690250
1,1000,25881,19771231,1977,INDL,C,D,STD,AE.2,000032102,USD,2.226,NaN,12,I,9.2500,9.2500,3089.0,1978,20.590500
2,1001,10015,19831231,1983,INDL,C,D,STD,AMFD.,000165100,USD,3.568,0.000,14,I,7.2500,7.2500,5812.0,1984,25.868000
3,1001,10015,19841231,1984,INDL,C,D,STD,AMFD.,000165100,USD,3.568,NaN,14,I,3.7500,3.7500,5812.0,1985,13.380000
4,1001,10015,19851231,1985,INDL,C,D,STD,AMFD.,000165100,USD,3.988,0.000,14,I,10.1250,10.1250,5812.0,1986,40.378500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276002,332115,80577,20201231,2020,INDL,C,D,STD,ARMP,04216R102,USD,18.688,14.444,12,A,2.9849,2.9849,2836.0,2021,55.781811
276003,335466,14756,20191231,2019,INDL,C,D,STD,HOFSQ,G4511M108,USD,25.662,NaN,19,A,1.0173,1.0173,4400.0,2020,26.105953
276004,339965,19654,20210131,2021,INDL,C,D,STD,SNOW,833445109,USD,287.918,237.946,11,A,281.4000,272.4500,7370.0,2022,81020.125200
276005,345920,20194,20201231,2020,INDL,C,D,STD,HYFM,44888K209,USD,33.500,NaN,14,A,52.5800,52.5800,3524.0,2021,1761.430000


In [4]:
yearly_xrd = ccm_fundamentals_annual.groupby(["LPERMNO","year"])["xrd","me"].mean().unstack()
yearly_xrd

/var/folders/y3/v6fyb_rs72bcn7ftc3lvwvmw0000gn/T/ipykernel_92786/2921738248.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  yearly_xrd = ccm_fundamentals_annual.groupby(["LPERMNO","year"])["xrd","me"].mean().unstack()


xrd                                               ...          me  \
year    1977 1978 1979 1980 1981 1982 1983 1984 1985 1986  ...        2013   
LPERMNO                                                    ...               
10000    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...         NaN   
10001    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    78.09210   
10002    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...         NaN   
10003    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...         NaN   
10005    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...         NaN   
...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...         ...   
93432    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...         NaN   
93433    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    19.16053   
93434    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    53.05956   
93435    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...         NaN   
93436    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  3868.42818   

                                                                           \
year             2014         2015         2016         2017         2018   
LPERMNO                                                                     
10000             NaN          NaN          NaN          NaN          NaN   
10001       83.929560    115.57776     78.26225    132.02600          NaN   
10002             NaN          NaN          NaN          NaN          NaN   
10003             NaN          NaN          NaN          NaN          NaN   
10005             NaN          NaN          NaN          NaN          NaN   
...               ...          ...          ...          ...          ...   
93432             NaN          NaN          NaN          NaN          NaN   
93433       16.114140      3.14358     44.97500     20.05885      8.18545   
93434       80.508800     46.56000     56.77588     78.48060     70.12200   
93435             NaN          NaN          NaN          NaN          NaN   
93436    18516.456039  27954.26808  31543.31425  34523.97009  52554.94595   

                                                      
year             2019        2020          2021 2022  
LPERMNO                                               
10000             NaN         NaN           NaN  NaN  
10001             NaN         NaN           NaN  NaN  
10002             NaN         NaN           NaN  NaN  
10003             NaN         NaN           NaN  NaN  
10005             NaN         NaN           NaN  NaN  
...               ...         ...           ...  ...  
93432             NaN         NaN           NaN  NaN  
93433        4.961642         NaN           NaN  NaN  
93434       44.060830     69.8838      97.95869  NaN  
93435             NaN         NaN           NaN  NaN  
93436    57442.278400  75717.7300  677443.20000  NaN  

[25331 rows x 92 columns]

Calculate $ RCD_t = f(XRD_t, XRD_{t-1}, XRD_{t-2}, XRD_{t-3}, XRD_{t-4}, XRD_{t-5}) / ME_t $

In [5]:
yearly_rcd = yearly_xrd["xrd"].apply(lambda row: (get_rnd(row)), axis = 1)
yearly_rcd = (yearly_rcd/yearly_xrd["me"]).T
yearly_rcd = yearly_rcd.stack()
yearly_rcd.name = "r&d"

yearly_rcd.to_csv("yearly_rcd.csv")
yearly_rcd

yearly_rcd.head()

year  LPERMNO
1981  10006      0.023672
      10057      0.049609
      10145      0.073265
      10153      0.259662
      10161      0.015693
Name: r&d, dtype: float64

In [6]:
# yearly_rcd_class = yearly_rcd.groupby(level=0).progress_apply(lambda x: assign_quantile(x))
# yearly_rcd_class = yearly_rcd_class.droplevel(0)
# yearly_rcd_class.name = "bin"
# yearly_rcd_class = yearly_rcd_class.astype("category").cat.reorder_categories(["Low","q2","q3","q4","High","N/A R&D"], ordered = True)
# yearly_rcd_class.to_csv("bins.csv")
# yearly_rcd_class.head()

  0%|          | 0/42 [00:00<?, ?it/s]

year  LPERMNO
1981  10006        q2
      10057        q3
      10145        q3
      10153      High
      10161       Low
Name: bin, dtype: category
Categories (6, object): ['Low' < 'q2' < 'q3' < 'q4' < 'High' < 'N/A R&D']

# Read CRSP Monthly Stock data

In [7]:
crsp_monthly_data = pd.read_csv("df_ms.csv", low_memory= False)
print("Parsing Dates")
crsp_monthly_data["date"] = crsp_monthly_data["date"].astype("str").progress_apply(lambda x: convert_to_date(x))

print("Add Year")
crsp_monthly_data["year"] = crsp_monthly_data["date"].progress_apply(lambda x: x.year)

print("Add Month")
crsp_monthly_data["month"] = crsp_monthly_data["date"].progress_apply(lambda x: x.month)

print("Add MarketCap")
crsp_monthly_data["mrktcap"] = abs(crsp_monthly_data['PRC'])*crsp_monthly_data['SHROUT']

crsp_monthly_data.drop_duplicates(["PERMNO", "date"], inplace= True)

crsp_monthly_data.set_index("PERMNO", inplace= True)
crsp_monthly_data

Parsing Dates


  0%|          | 0/3823198 [00:00<?, ?it/s]

Add Year


  0%|          | 0/3823198 [00:00<?, ?it/s]

Add Month


  0%|          | 0/3823198 [00:00<?, ?it/s]

Add MarketCap


,date,SHRCD,SICCD,TICKER,SHRCLS,PRC,RET,SHROUT,CFACPR,CFACSHR,year,month,mrktcap
PERMNO,,,,,,,,,,,,,
10000,1985-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1985,12,NaN
10000,1986-01-31,10.0,3990,OMFGA,A,-4.37500,C,3680.0,1.0,1.0,1986,1,1.610000e+04
10000,1986-02-28,10.0,3990,OMFGA,A,-3.25000,-0.257143,3680.0,1.0,1.0,1986,2,1.196000e+04
10000,1986-03-31,10.0,3990,OMFGA,A,-4.43750,0.365385,3680.0,1.0,1.0,1986,3,1.633000e+04
10000,1986-04-30,10.0,3990,OMFGA,A,-4.00000,-0.098592,3793.0,1.0,1.0,1986,4,1.517200e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...
93436,2020-08-31,11.0,9999,TSLA,NaN,498.32001,0.741452,931809.0,1.0,1.0,2020,8,4.643391e+08
93436,2020-09-30,11.0,9999,TSLA,NaN,429.01001,-0.139087,948000.0,1.0,1.0,2020,9,4.067015e+08
93436,2020-10-30,11.0,9999,TSLA,NaN,388.04001,-0.095499,947901.0,1.0,1.0,2020,10,3.678235e+08


In [8]:
# Filter for Share class = A or missing
crsp_monthly_data = crsp_monthly_data[(crsp_monthly_data["SHRCLS"] == "A")|(crsp_monthly_data["SHRCLS"].isna())]

# Filter for Share Code = 10 or 11
crsp_monthly_data = crsp_monthly_data[(crsp_monthly_data["SHRCD"] == 10)|(crsp_monthly_data["SHRCD"] == 11)]

# Remove Financial Companies
crsp_monthly_data = crsp_monthly_data[pd.to_numeric(crsp_monthly_data["SICCD"], errors= "coerce").notnull()]
crsp_monthly_data["SICCD"] = crsp_monthly_data["SICCD"].astype("int")
crsp_monthly_data = crsp_monthly_data[(crsp_monthly_data["SICCD"] < 6000)|(crsp_monthly_data["SICCD"] > 6999)]

# B in RET occours when none of the PERMNO entries have any price. Thus no return can exist. Thus, dropping
crsp_monthly_data = crsp_monthly_data.drop(crsp_monthly_data[crsp_monthly_data["RET"] == "B"].index).dropna(subset=["RET"])#.reset_index(drop = True)

# C in RET occours when the previous price does not exist. thus returns can not be calculated.
crsp_monthly_data["RET"] = crsp_monthly_data["RET"].replace("C",0.0).astype("float")

crsp_monthly_data

,date,SHRCD,SICCD,TICKER,SHRCLS,PRC,RET,SHROUT,CFACPR,CFACSHR,year,month,mrktcap
PERMNO,,,,,,,,,,,,,
10000,1986-01-31,10.0,3990,OMFGA,A,-4.37500,0.000000,3680.0,1.0,1.0,1986,1,1.610000e+04
10000,1986-02-28,10.0,3990,OMFGA,A,-3.25000,-0.257143,3680.0,1.0,1.0,1986,2,1.196000e+04
10000,1986-03-31,10.0,3990,OMFGA,A,-4.43750,0.365385,3680.0,1.0,1.0,1986,3,1.633000e+04
10000,1986-04-30,10.0,3990,OMFGA,A,-4.00000,-0.098592,3793.0,1.0,1.0,1986,4,1.517200e+04
10000,1986-05-30,10.0,3990,OMFGA,A,-3.10938,-0.222656,3793.0,1.0,1.0,1986,5,1.179388e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...
93436,2020-08-31,11.0,9999,TSLA,NaN,498.32001,0.741452,931809.0,1.0,1.0,2020,8,4.643391e+08
93436,2020-09-30,11.0,9999,TSLA,NaN,429.01001,-0.139087,948000.0,1.0,1.0,2020,9,4.067015e+08
93436,2020-10-30,11.0,9999,TSLA,NaN,388.04001,-0.095499,947901.0,1.0,1.0,2020,10,3.678235e+08


**Assumption:**

Not discarding years with <12 month returns

In [9]:
crsp_monthly_avg_returns = crsp_monthly_data.reset_index()[["PERMNO", "year", "month", "RET", "mrktcap"]].fillna(0).groupby(["year", "month", "PERMNO"])[["RET","mrktcap"]].agg("mean")
crsp_monthly_avg_returns = crsp_monthly_avg_returns.reset_index().rename({"PERMNO":"LPERMNO"}, axis = 1)
crsp_monthly_avg_returns

,year,month,LPERMNO,RET,mrktcap
0,1976,1,10006,0.211039,2.657625e+05
1,1976,1,10014,0.400000,1.317488e+04
2,1976,1,10050,0.296296,7.000000e+03
3,1976,1,10057,0.174603,4.079250e+04
4,1976,1,10058,0.272727,2.523500e+03
...,...,...,...,...,...
2073750,2020,12,93422,0.484472,5.806170e+05
2073751,2020,12,93423,0.109665,2.897716e+06
2073752,2020,12,93426,0.076239,3.951370e+05
2073753,2020,12,93434,0.122605,9.810226e+04


# Sorted Portfolio Analyisis Tables (R&D)

## Helper Functions

In [63]:
def get_summary(data: pd.DataFrame, mode: str = "equal", verbose: bool = False)->pd.DataFrame:
    
    monthly_rets_data = pd.DataFrame(dtype= np.float64)
    if mode == "equal":
        
        monthly_rets_data = data.groupby(["year","month","bin"])["RET"].mean().unstack()
    elif mode == "value":
        monthly_rets_data = data.groupby(["year","month","bin"])[["RET","mrktcap"]].apply(lambda x: (
                                                                                                    (x['RET'].multiply(x['mrktcap'])/x["mrktcap"].sum()).sum()
                                                                                                )).unstack()

    summary_df = pd.DataFrame(dtype= np.float64, 
                                index = ["All-time","Pre-2000", "2000-2012","2013-2020"], 
                                columns=pd.CategoricalIndex(["Low","q2","q3","q4","High","N/A R&D"]))

    summary_df.iloc[0] = monthly_rets_data[:2012].mean()
    summary_df.iloc[1] = monthly_rets_data.loc[:1999].mean()
    summary_df.iloc[2] = monthly_rets_data.loc[2000:2012].mean()
    summary_df.iloc[3] = monthly_rets_data.loc[2013:].mean()

    if verbose:
        summary_df.T.plot(figsize = (15,10), title = "Average Returns by Bins", ylabel = "Avg. Returns")
        # ((monthly_rets_data + 1).cumprod() - 1).plot(figsize = (15,10), title = "Monthly Returns by Quantile", ylabel = "Avg. Returns")
        (((monthly_rets_data.groupby(level = 0).mean()) + 1).cumprod() - 1).plot(figsize = (15,10), title = "Annual Returns by Quantile", ylabel = "Avg. Returns")
    
    return monthly_rets_data, summary_df

In [64]:
df_merged = pd.merge(yearly_rcd, crsp_monthly_avg_returns, how = 'inner', on = ['LPERMNO','year'], validate='1:m')[["year", "month", "LPERMNO", "r&d", "RET", "mrktcap"]]
df_merged

,year,month,LPERMNO,r&d,RET,mrktcap
0,1981,1,10006,0.023672,0.108696,4.594590e+05
1,1981,2,10006,0.023672,-0.022059,4.493239e+05
2,1981,3,10006,0.023672,-0.005013,4.414410e+05
3,1981,4,10006,0.023672,-0.066327,4.121618e+05
4,1981,5,10006,0.023672,0.066995,4.335581e+05
...,...,...,...,...,...,...
650968,2020,8,93436,0.034630,0.741452,4.643391e+08
650969,2020,9,93436,0.034630,-0.139087,4.067015e+08
650970,2020,10,93436,0.034630,-0.095499,3.678235e+08
650971,2020,11,93436,0.034630,0.462736,5.380286e+08


## All Stocks

### Merge Datasets

In [65]:
bins = df_merged[["year","LPERMNO","r&d"]].groupby(["year","LPERMNO"])["r&d"].mean()
bins = bins.groupby(level= 0).progress_apply(lambda x: assign_quantile(x)).astype("category").cat.reorder_categories(["Low","q2","q3","q4","High","N/A R&D"], ordered = True).droplevel(0)#.reset_index()
bins.name = "bin"


df_binned = pd.merge(bins, df_merged, how = 'inner', on = ['LPERMNO','year'], validate='1:m')
df_binned

  0%|          | 0/40 [00:00<?, ?it/s]

,LPERMNO,year,bin,month,r&d,RET,mrktcap
0,10006,1981,q2,1,0.023672,0.108696,459459.000
1,10006,1981,q2,2,0.023672,-0.022059,449323.875
2,10006,1981,q2,3,0.023672,-0.005013,441441.000
3,10006,1981,q2,4,0.023672,-0.066327,412161.750
4,10006,1981,q2,5,0.023672,0.066995,433558.125
...,...,...,...,...,...,...,...
650968,93423,2020,N/A R&D,8,0.000000,0.249569,1841878.260
650969,93423,2020,N/A R&D,9,0.000000,-0.065808,1720668.600
650970,93423,2020,N/A R&D,10,0.000000,0.065025,1837202.740
650971,93423,2020,N/A R&D,11,0.000000,0.421369,2611343.210


### Equally weighted

In [66]:
# monthly_rets_eq, summary_eq = get_summary(data= df_binned, mode= "equal", verbose= True)
monthly_rets_eq, summary_eq = get_summary(data= df_binned, mode= "equal")

summary_eq

,Low,q2,q3,q4,High,N/A R&D
All-time,0.009130,0.011672,0.014104,0.015579,0.024040,0.013752
Pre-2000,0.009673,0.012017,0.014699,0.016526,0.025266,0.014510
2000-2012,0.007056,0.009594,0.011488,0.013078,0.023898,0.014584
2013-2020,0.011212,0.014233,0.016941,0.017396,0.021357,0.010598


### Value - Weighted

In [67]:
# monthly_rets_val, summary_val = get_summary(data= df_binned, mode= "value", verbose= True)
monthly_rets_val, summary_val = get_summary(data= df_binned, mode= "value")

summary_val

,Low,q2,q3,q4,High,N/A R&D
All-time,0.014242,0.017009,0.017622,0.022515,0.029255,0.017940
Pre-2000,0.017667,0.020040,0.019771,0.023457,0.025309,0.023510
2000-2012,0.009241,0.010161,0.013333,0.021169,0.030957,0.011084
2013-2020,0.014235,0.020941,0.019488,0.022464,0.035862,0.015852


## W/o Top 1000 stocks

### Merge Datasets

In [68]:
def leave_1000(data: pd.DataFrame, filter_top: int = 1000):
    # print(data["year"].unique())
    
    avg_mrktcap = (data[["LPERMNO","month", "mrktcap"]].groupby("LPERMNO")["mrktcap"].mean()).sort_values(ascending= False)
    if avg_mrktcap.shape[0] >= filter_top:
        threshold = avg_mrktcap.iloc[filter_top - 1]
    else:
        threshold = avg_mrktcap.iloc[0]
    
    # print(avg_mrktcap.shape, threshold)
    return threshold

In [69]:
upper_bound = df_merged.groupby(["year"]).apply(lambda x: leave_1000(x))
upper_bound.name = "upper_bound"

df_minus_1000 = pd.merge(df_merged, upper_bound, how = "left", on="year", validate="m:1")
df_minus_1000 = df_minus_1000[df_minus_1000["mrktcap"] <= df_minus_1000["upper_bound"]]
df_minus_1000

,year,month,LPERMNO,r&d,RET,mrktcap,upper_bound
0,1981,1,10006,0.023672,0.108696,459459.000,4.393483e+07
1,1981,2,10006,0.023672,-0.022059,449323.875,4.393483e+07
2,1981,3,10006,0.023672,-0.005013,441441.000,4.393483e+07
3,1981,4,10006,0.023672,-0.066327,412161.750,4.393483e+07
4,1981,5,10006,0.023672,0.066995,433558.125,4.393483e+07
...,...,...,...,...,...,...,...
650956,2020,8,93434,0.100746,0.008584,78567.550,1.556946e+05
650957,2020,9,93434,0.100746,0.055319,82958.480,1.556946e+05
650958,2020,10,93434,0.100746,-0.080645,76307.040,1.556946e+05
650959,2020,11,93434,0.100746,0.144737,87388.020,1.556946e+05


In [70]:
bins = df_minus_1000[["year","LPERMNO","r&d"]].groupby(["year","LPERMNO"])["r&d"].mean()
bins = bins.groupby(level= 0).progress_apply(lambda x: assign_quantile(x)).astype("category").cat.reorder_categories(["Low","q2","q3","q4","High","N/A R&D"], ordered = True).droplevel(0)#.reset_index()
bins.name = "bin"


df_minus_1000_binned = pd.merge(bins, df_minus_1000, how = 'inner', on = ['LPERMNO','year'], validate='1:m')
df_minus_1000_binned

  0%|          | 0/40 [00:00<?, ?it/s]

,LPERMNO,year,bin,month,r&d,RET,mrktcap,upper_bound
0,10006,1981,q2,1,0.023672,0.108696,459459.000,4.393483e+07
1,10006,1981,q2,2,0.023672,-0.022059,449323.875,4.393483e+07
2,10006,1981,q2,3,0.023672,-0.005013,441441.000,4.393483e+07
3,10006,1981,q2,4,0.023672,-0.066327,412161.750,4.393483e+07
4,10006,1981,q2,5,0.023672,0.066995,433558.125,4.393483e+07
...,...,...,...,...,...,...,...,...
196207,93373,2020,N/A R&D,8,0.000000,0.099010,71549.490,1.556946e+05
196208,93373,2020,N/A R&D,9,0.000000,-0.450450,39551.180,1.556946e+05
196209,93373,2020,N/A R&D,10,0.000000,0.001639,39616.018,1.556946e+05
196210,93373,2020,N/A R&D,11,0.000000,1.487725,98553.760,1.556946e+05


### Equally weighted

In [71]:
# monthly_rets_eq_minus_1000, summary_eq_minus_1000 = get_summary(data= df_minus_1000_binned, mode= "equal", verbose= True)
monthly_rets_eq_minus_1000, summary_eq_minus_1000 = get_summary(data= df_minus_1000_binned, mode= "equal")

summary_eq_minus_1000

,Low,q2,q3,q4,High,N/A R&D
All-time,-0.007580,-0.003860,0.000524,-0.000476,0.017013,0.002706
Pre-2000,-0.012433,-0.006326,0.001980,-0.000929,0.020533,0.002269
2000-2012,-0.004906,-0.001522,-0.001020,0.004043,0.018455,0.007780
2013-2020,-0.000403,-0.001803,-0.000425,-0.006740,0.006309,-0.004501


### Value - Weighted

In [72]:
# monthly_rets_val_minus_1000, summary_val_minus_1000 = get_summary(data= df_minus_1000_binned, mode= "value", verbose= True)
monthly_rets_val_minus_1000, summary_val_minus_1000 = get_summary(data= df_minus_1000_binned, mode= "value")

summary_val_minus_1000

,Low,q2,q3,q4,High,N/A R&D
All-time,-0.001894,0.002472,0.008948,0.010956,0.037522,0.009155
Pre-2000,-0.007412,-0.001313,0.009359,0.007866,0.031495,0.008983
2000-2012,0.001124,0.005494,0.008984,0.015153,0.038843,0.013660
2013-2020,0.006307,0.006548,0.007916,0.011473,0.049686,0.002241


# Portfolio Performance

## Read Fama French Monthly Data

In [107]:
ff_dataset = pd.read_csv("df_ff.csv")
ff_dataset["Date"] = ff_dataset["Date"].progress_apply(lambda x: convert_to_date(str(x)+"01"))
ff_dataset["year"] = ff_dataset["Date"].apply(lambda x: x.year)
ff_dataset["month"] = ff_dataset["Date"].apply(lambda x: x.month)
ff_dataset = ff_dataset.rename({"Mkt-RF":"mkt_risk_premium"}, axis= 1)
ff_dataset

  0%|          | 0/1143 [00:00<?, ?it/s]

,Date,mkt_risk_premium,SMB,HML,RF,year,month
0,1926-07-01,2.96,-2.38,-2.73,0.22,1926,7
1,1926-08-01,2.64,-1.47,4.14,0.25,1926,8
2,1926-09-01,0.36,-1.39,0.12,0.23,1926,9
3,1926-10-01,-3.24,-0.13,0.65,0.32,1926,10
4,1926-11-01,2.53,-0.16,-0.38,0.31,1926,11
...,...,...,...,...,...,...,...
1138,2021-05-01,0.29,-0.28,7.04,0.00,2021,5
1139,2021-06-01,2.75,1.80,-7.76,0.00,2021,6
1140,2021-07-01,1.27,-3.96,-1.75,0.00,2021,7
1141,2021-08-01,2.90,-0.48,-0.13,0.00,2021,8


## Helper Function

In [180]:
def func(rets_data: pd.DataFrame, ff_data: pd.DataFrame = ff_dataset, verbose: bool = False, show_capm_plot: bool = False)-> pd.Series:
    rets_data = rets_data.copy()
    rets_data["rnd_investment_risk_premium"] = rets_data["q4"] - rets_data["q2"]
    # rets_data["rnd_investment_risk_premium"] = rets_data["High"] - rets_data["Low"]
    
    rets_data = pd.merge(left= rets_data, right= ff_data, on=["year", "month"], validate= "m:1")[["rnd_investment_risk_premium","mkt_risk_premium","SMB","HML"]]
    
    # CAPM
    formula = 'rnd_investment_risk_premium ~ mkt_risk_premium'
    y, X = dmatrices(formula, data=rets_data, return_type= "dataframe")
    model = sm.OLS(y, X)
    result = model.fit()
    alpha_capm, beta_capm = result.params

    # FF
    formula = 'rnd_investment_risk_premium ~ mkt_risk_premium + SMB + HML'
    y, X_ff = dmatrices(formula, data=rets_data, return_type= "dataframe")
    model_ff = sm.OLS(y, X_ff)
    result_ff = model_ff.fit()
    alpha_ff, beta_ff_mrp, beta_ff_smb, beta_ff_hml = result_ff.params

    if verbose:
        print("CAPM:\t"+formula)
        print(result.summary(),end="\n––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––\n\n")
    
        print("FF:\t"+formula)
        print(result_ff.summary(),end="\n––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––\n\n")
    
    if show_capm_plot:
        try:
            plt.figure(figsize=(10,7))
            plt.scatter(X["mkt_risk_premium"], y, label = "R&D Investment Risk Premium")
            plt.legend()
            plt.xlabel("Systemic Risk")
            plt.ylabel("Risk Premium")
            plt.title("Security Characteristics Line Regression: R&D Investment Risk Premium (β: "+str(round(beta_capm, 5))+")")
            plt.axhline(y=0, color='k', linewidth = 0.7)
            plt.axvline(x=0, color='k', linewidth = 0.7)

            f_x = np.dot(X,np.asarray(result.params))
            plt.plot(X["mkt_risk_premium"],f_x, color = "red", linestyle = "dashed", linewidth = 0.7)
        except:
            print(sys.exc_info())
            raise Exception
    


    res = pd.Series([alpha_capm, alpha_ff, ( ( (y.mean()[0]) * 12 ) / ( (y.std()[0]) * np.sqrt(12) ) )],
                        index=["alpha_CAPM", "alpha_FF", "Annualised sharpe"])
    return res

## Portfolio Upto 2012:

In [181]:

pd.DataFrame(
    {
        "Equally weighted; All Stocks"          : func(monthly_rets_eq.loc[:2012]),
        "Value weighted; All Stocks"            : func(monthly_rets_val.loc[:2012]),
        "Equally weighted; w/o top 1000 Stocks" : func(monthly_rets_eq_minus_1000.loc[:2012]),
        "Value weighted; w/o top 1000 Stocks"   : func(monthly_rets_val_minus_1000.loc[:2012])
    }
).T
# func(monthly_rets_eq, verbose= True)

,alpha_CAPM,alpha_FF,Annualised sharpe
Equally weighted; All Stocks,0.003362,0.003424,0.515789
Value weighted; All Stocks,0.005647,0.004977,0.653908
Equally weighted; w/o top 1000 Stocks,0.005145,0.005717,0.381849
Value weighted; w/o top 1000 Stocks,0.008823,0.009655,0.559206


## Portfolio Extended Period (till 2020):

In [182]:
pd.DataFrame(
    {
        "Equally weighted; All Stocks"          : func(monthly_rets_eq),
        "Value weighted; All Stocks"            : func(monthly_rets_val),
        "Equally weighted; w/o top 1000 Stocks" : func(monthly_rets_eq_minus_1000),
        "Value weighted; w/o top 1000 Stocks"   : func(monthly_rets_val_minus_1000)
    }
).T
# func(monthly_rets_eq, verbose= True)

,alpha_CAPM,alpha_FF,Annualised sharpe
Equally weighted; All Stocks,0.003073,0.003254,0.517970
Value weighted; All Stocks,0.004814,0.004597,0.576364
Equally weighted; w/o top 1000 Stocks,0.002733,0.003296,0.227575
Value weighted; w/o top 1000 Stocks,0.007650,0.008472,0.486017


***
# END OF CODE
***